# Simple solution of a modified Eikonal equation
We solve a modified version of the Eikonal equation. The mesh should contain a boundary group labeled `inlet` to locate the inlet boundary condition.

## Import mesh to create topology and geometry
The mesh can be a 1D, 2D or 3D mesh. The elements must be linear and there must be a boundary named `inlet`. 

In [72]:
import os 
import numpy as np

from nutils import function, mesh, solver, export
from nutils.expression_v2 import Namespace

# Nutils namespace
ns = Namespace()

# Geometry
mesh_name = os.path.join("meshes", "box.msh")
domain, geom = mesh.gmsh(mesh_name)

# Define the geometry variable x as well as gradients, normal and jacobians on the domain.
ns.x = geom
ns.define_for("x", gradient="∇", normal="n", jacobians=("dV", "dS"))

gmsh > loaded 3d gmsh topology consisting of #1119 elements
- volume groups: domain #1119
- boundary groups: inlet #90


We sample the domain to get a tuple of all node positions in the mesh `X` and compute a bounding box. The bounding box is then use to define the parameter `l_ref` and consequently the value of $G_0$.

In [73]:
# Get bounding box of input mesh
bezier = domain.sample("vtk", 2) 
X = np.array(bezier.eval(["x_i"] @ ns))
X_min = np.min(X, axis=1)
X_max = np.max(X, axis=1)
l_ref = np.max(X_max-X_min)

# Parameters in namesapce
ns.G0 = 2.0 / l_ref
ns.σ = 0.1

## Basis
A *nutils* basis is a vector-based function that evalautes at any given point $\mathbf{x}$ on the domain to the array of basis functions.

In [74]:
ns.basis = domain.basis("std", degree=1)

## Problem formulation

The problem to be solved is 
$$
\nabla G(\mathbf{x}) \cdot  \nabla  G(\mathbf{x}) + \sigma G(\mathbf{x}) \Delta G(\mathbf{x}) = (1 + 2\sigma)G^4(\mathbf{x}) \quad \mathbf{x} \in \Omega
$$
for the inverse wall distance $G(\mathbf{x})$. The weak form of this equation is
$$
(1-\sigma) \int_\Omega \nabla G(\mathbf{x}) \cdot  \nabla G(\mathbf{x}) H(\mathbf{x}) dV - \sigma \int_V G(\mathbf{x}) \nabla G(\mathbf{x}) \cdot \nabla H(\mathbf{x}) dV - (1+2\sigma) \int_\Omega G^4(\mathbf{x}) H(\mathbf{x}) dV = 0 \quad \forall H
$$
assuming $\nabla G(\mathbf{x}) = 0$ on $\partial \Omega\setminus\partial\Omega_{inlet}$.

In [75]:
ns.G = function.dotarg("lhs", ns.basis)
res = domain.integral("(1 - σ) ∇_i(G) ∇_i(G) basis_n dV" @ ns, degree=2)
res -= domain.integral("σ G ∇_i(G) ∇_i(basis_n) dV" @ ns, degree=2)
res -= domain.integral("(1 + 2 σ) G^4 basis_n dV" @ ns, degree=2)

## Boundary condition at inlet
The Dirichlet boundary condition is 
$$
G(\mathbf{x}) = G_0 \quad \mathbf{x} \in \partial \Omega_{inlet}.
$$

It is expressed as 
$$ 
 \underset{\mathbf{G}}{\min} \int_{\partial\Omega} (G - G_0)^2 dS = 0 \quad \text{at} \quad \partial\Omega_\text{inlet}
$$

In [76]:
sqr = domain.boundary["inlet"].integral("(G - G0)^2 dS" @ ns, degree=2)
cons = solver.optimize("lhs", sqr, droptol=1e-15)

optimize > solve > solving 58 dof system to machine precision using arnoldi solver
optimize > solve > solver returned with residual 6e-17
optimize > constrained 58/338 dofs
optimize > optimum value 0.00e+00


## Solve the problem

In [77]:
# Initial values set to G_0
sqr = domain.integral("(G - G0)^2 dV" @ ns, degree=2)
lhs0 = solver.optimize("lhs", sqr, droptol=1e-15)

# Solve non-linear equation with Newton solver
lhs = solver.newton("lhs", res, constrain=cons, lhs0=lhs0).solve(tol=1E-10)

optimize > solve > solving 338 dof system to machine precision using arnoldi solver
optimize > solve > solver returned with residual 3e-17
optimize > constrained 338/338 dofs
optimize > optimum value 0.00e+00
newton 0% > solve > solving 280 dof system to tolerance 1e-03 using arnoldi solver
newton 0% > solve > solver returned with residual 9e-16
newton 0% > estimated residual minimum at inf% of update vector
newton 0% > update accepted at relaxation 1.0
newton 5% > solve > solving 280 dof system to tolerance 4e-04 using arnoldi solver
newton 5% > solve > solver returned with residual 4e-16
newton 5% > estimated residual minimum at 119% of update vector
newton 5% > update accepted at relaxation 1
newton 11% > solve > solving 280 dof system to tolerance 1e-04 using arnoldi solver
newton 11% > solve > solver returned with residual 1e-16
newton 11% > estimated residual minimum at 98% of update vector
newton 11% > update accepted at relaxation 1
newton 16% > solve > solving 280 dof system t

## Postprocessing
The resulting vector is interpolated using beziers. The actual distance $D$ is then computed from the inverse wall distance $G$ by
$$
D(\mathbf{x}) = \frac{1}{G(\mathbf{x})} - \frac{1}{G_0}.
$$

In [78]:
ns.D = "1 / G  - 1 / G0"
bezier = domain.sample("vtk", 2) 
x, G, D = bezier.eval(["x_i", "G", "D"] @ ns, lhs=lhs)
export.vtk(mesh_name[:-4], bezier.tri, x, G=G, D=D)

meshes/box.vtk
